# Piotroski F-Score 

Main metrics:
- Profitability:
    - Return On Assets
    - Cash Flow from Operations
- Leverage, Liquidity, and Source of Funds:
  - Dilution Ratio
  - Current Ratio
  - Long Term Debt To Equity Ratio
- Operating Efficiency:
  - Gross Margin
  - Asset Turnover

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import yahoo_fin.stock_info as si



In [2]:
si.get_stats_valuation("MSFT")

,0,1
0,Market Cap (intraday),2.09T
1,Enterprise Value,2.05T
2,Trailing P/E,30.85
3,Forward P/E,25.64
4,PEG Ratio (5 yr expected),2.09
5,Price/Sales (ttm),10.20
6,Price/Book (mrq),11.29
7,Enterprise Value/Revenue,10.04
8,Enterprise Value/EBITDA,20.75


In [3]:
tick = yf.Ticker("MSFT")

In [ ]:
tick.financials

In [ ]:
# Download data
tech_tickers = ["AAPL", "MSFT", "AMZN", "GOOG"]
bank_tickers = ["JPM", "BAC", "GS", "MS"]
finance_tickers = ["MA", "AXP"]

tech_data = yf.download(tech_tickers, start="2015-01-01", end="2023-03-01")["Adj Close"]
finance_data = yf.download(finance_tickers, start="2015-01-01", end="2023-03-01")[
    "Adj Close"
]
bank_data = yf.download(bank_tickers, start="2015-01-01", end="2023-03-01")["Adj Close"]

# Combine data into a single DataFrame
data = pd.concat([tech_data, finance_data, bank_data], axis=1)
data.columns = ["AAPL", "MSFT", "AMZN", "GOOG", "JPM", "BAC", "GS", "MS", "MA", "AXP"]
data


In [ ]:
df = yf.download(['AAPL',"MS"], start="2015-01-01", end="2023-03-01")

In [ ]:
def piotroski_score(financial_data):
    score = 0
    if financial_data['ROA'] > 0:
        score += 1
    if financial_data['CFO'] > 0:
        score += 1
    if financial_data['DILUTION_RATIO'] < 1:
        score += 1
    if financial_data['ASSET_TURNOVER'] > financial_data['ASSET_TURNOVER'].shift(1):
        score += 1
    if financial_data['CURRENT_RATIO'] > financial_data['CURRENT_RATIO'].shift(1):
        score += 1
    if financial_data['LONG_TERM_DEBT_TO_EQUITY_RATIO'] < financial_data['LONG_TERM_DEBT_TO_EQUITY_RATIO'].shift(1):
        score += 1
    if (financial_data['GROSS_MARGIN'] - financial_data['GROSS_MARGIN'].shift(1)) > 0:
        score += 1
    if financial_data['RETURN_ON_ASSETS'] > financial_data['RETURN_ON_ASSETS'].shift(1):
        score += 1
    return score

In [ ]:
# Calculate the ROA
final_df['roa'] = financial_data['net_income'] / financial_data['total_assets']

final_df['cash_flow_from_operations'] = financial_data['net_income'] + financial_data['depreciation_and_amortization'] - financial_data['change_in_working_capital']

final_df['dilution_ratio'] = (financial_data['common_stock_issued'] / financial_data['shares_outstanding']) + 1

final_df['asset_turnover'] = financial_data['revenue'] / (financial_data.groupby(pd.Grouper(freq='1Y'))['total_assets'].transform(lambda x: x.first('Q')).ffill() +\
                            financial_data.groupby(pd.Grouper(freq='1Y'))['total_assets'].transform(lambda x: x.last('Q')).ffill())/2


final_df['current_ratio'] = financial_data['total_current_assest']/financial_data['total_current_liabilities']

final_df['long_term_debt_to_equity'] = financial_data['long_term_debt'] / financial_data['total_equity']

final_df['gross_margin'] = (financial_data['revenue']-financial_data['cost_of_revenue'])/financial_data['revenue']

final_df['asset_turnover_lag_1'] = final_df['asset_turnover'].shift(1)

final_df['current_ratio_lag_1'] = final_df['current_ratio'].shift(1)

final_df['long_term_debt_to_equity_lag_1'] = final_df['long_term_debt_to_equity'].shift(1)

final_df['gross_margin_lag_1'] = final_df['gross_margin'].shift(1)

final_df['roa_lag_1'] = final_df['roa'].shift(1)

In [ ]:
final_df['piotroski_score'] = final_df.apply(lambda x: piotroski_score(x), axis=1)

# Fundamental analysis

## Fundamental ratios
We will explore four ratios that are the most popular and widely used and can be used as the first line of analysis to get the pulse of the company’s financials.

**Price to earnings ratio (P/E)**
It measures the company’s current share price relative to its per-share earnings. A higher value would mean overvalued and a lower value means undervalued. This doesn’t mean any stock with a higher PE ratio is overvalued or the other way. The PE ratio should be used along with other valuation factors. In short, it is the premium that an investor is willing to pay for a unit of earnings. As a thumb rule, PE greater than 30 is considered high.

$PE = Stock price / Earnings per share (EPS)$

**Price to book ratio (P/B)**
It is the measure of the relation between the price and the book value. A lower PB ratio indicates undervalued stock. For example, if the book value is 50 and the stock price is 500 then the PB will be 10 meaning PB is 10 times the book value.

$PB = stock price / Book value$

**Price to sales ratio (P/S)**
The PS is the easiest ratio to get a valuation of a company where an investor knows exactly the price they are paying for the company. It indicates, the premium the investor is willing to pay for every unit of sales per stock. A low value would mean undervalued and a value higher than average means overvalued.

$PS = stock price / sales per share$

**Price-Earning to growth ratio (PEG)**
The PEG ratio is an advanced version of the PE ratio as it also factors in the expected growth. It provides a better picture compared to the PE ratio. A value of fewer than 1 means undervalued.

$PEG = PE/EPSGrowth$



In [ ]:
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as si
import pandas as pd

Over_under = PE of a specific stock / mean PE of all the stocks

If the value of over_under is greater than 1 then the stock is overvalued
If the value of over_under is less than 1 then the stock is undervalued
If the value of over_under is 1 then it is fairly valued

In [ ]:

class FundamentalRatio:
    
    def __init__(self, start_date, end_date, tickers, ratio_stat):
        
        self.start_date = start_date
        self.end_date = end_date
        self.tickers = tickers
        self.ratio_stat = ratio_stat
    
    
    def load_data(self):
        try:            
            data = pd.DataFrame(columns = self.tickers)
            for ticker in self.tickers:
                data[ticker] = yf.download(self.ticker, 
                                        self.start_date,
                                        self.end_date)['Adj Close']
            
            return data
        except Exception as e:
            print(f'An exception occurred while executing load_data: {e}')        
    

    def get_fundamental(self):
        
        try:
            df_fundamentals = pd.DataFrame()
            
            for ticker in range(len(self.tickers)):
                stock_name = str(self.tickers[ticker])
                fundamental_ratio = si.get_stats_valuation(stock_name)
                fundamental_ratio.index = fundamental_ratio[0]
                fundamental_ratio = fundamental_ratio.drop(labels=0,axis=1)
                tmp_table = fundamental_ratio.T
                tmp_table = tmp_table[self.ratio_stat]
                df_fundamentals = df_fundamentals.append(tmp_table)

            df_fundamentals.index = self.tickers
            df_fundamentals = df_fundamentals.astype('float')
            # df_fundamentals = pd.to_numeric(df_fundamentals['Trailing P/E'])
            df_fundamentals.dropna(inplace = True)
            return df_fundamentals
        except Exception as e:
            print(f'An exception occurred while executing get_fundamental: {e}')        


    def get_over_under_stocks(self):
        
        try:        
            df_fundamentals = self.get_fundamental()
            df_fundamentals['Trailing P/E'].mean()
            df_fundamentals['over_under'] = (df_fundamentals['Trailing P/E']) / (df_fundamentals['Trailing P/E'].mean())
           
            category = []
            for i in df_fundamentals['over_under']:
                if i < 1: category.append('Under Valued')
                elif i>1: category.append('Over Valued')
                else: category.append('Fair Valued')
            
            df_fundamentals['Category'] = category
            return df_fundamentals
        except Exception as e:
            print(f'An exception occurred while executing get_fundamental: {e}')   
    

##################################################################
# Initiate execution
##################################################################
start_date = '2022-02-02'
end_date = '2023-02-02'

tickers = ['AAPL', 'IBM', 'MSFT', 'WMT', 'AMGN', 'AXP', 'BA','NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']

ratio_stat = ['Trailing P/E','Forward P/E','PEG Ratio (5 yr expected)', 'Price/Book (mrq)',
              'Price/Sales (ttm)','Enterprise Value/EBITDA', 'Enterprise Value/Revenue']

obj_fundamentals = FundamentalRatio(start_date, end_date, tickers, ratio_stat)
# obj_fundamentals.get_fundamental()

df_classify = obj_fundamentals.get_over_under_stocks()
df_classify

# Filter the under valued stocks
df_classify[df_classify['Category'] == 'Under Valued']

# Alternative way

Market_Cap — the total market value of the company’s outstanding shares
Price — the current market price of the share
High — 52-week high of the share
Low — 52-week low of the share
PE — Price to Earnings ratio
ROE — Return on Equity
ROCE — Return on Capital Employed
Dividend — Dividend yield of the company



PE — Price-to-Earnings Ratio: It measures the current share price relative to its earnings per share

ROE — Return on Equity: It is the ratio of net income to the shareholder equity.

ROCE — Return on Capital Employment: ROCE is the ratio of the operating profits of a company to its total capital employed.

PE + ROE + ROCE:

PE ratio is normally not used in isolation. It is either compared with other companies from the same industry (Eg: PE of INFY is compared with the PE of HCL, LTI, TCS etc) or compared with the industry PE (PE of Infy is compared with PE of IT sector)
Higher PE indicates that the stock is priced highly. Lower PE indicates a good investment opportunity (provided the other fundamentals of the company is good)
High ROE indicates that the company is good in converting its earnings into profits.
A higher ROCE indicates that the company is generating higher returns for the debt holders than for the equity holders. The higher the value of the ROCE ratio, the better are the chances of profits.
ROE considers the returns from equity shareholder’s point of view only whereas ROCE considers the debt and other liabilities as well. This provides a better indication of financial performance for companies with significant debt.
If the ROCE value is higher than the ROE value, it implies that the company is efficiently using its debts to reduce the cost of capital.
Mr. Warren Buffet, one of the most successful investors of the 20th century, prefers companies where the ROE and ROCE values are almost close to each other and both are above 20%



## Valuation methods

The valuation methods to pick and choose by asset class:

1) Discounted-Cash-Flow

Any asset with clear value capture, a bounded range of outcomes, and investing style long enough to realize that value.


2) Price-to-Earnings

Ultimately a DCF proxy in the following way:
Oversimplifying, the terminal value of a DCF = profit level divided by: the discount rate minus the long term growth rate.
The discount rate minus the growth rate determines the 'terminal multiple.'

E.g. if you think profitability grows 3%/yr and use a discount rate of 10%, the multiple is: 1 / (10%-3%) = a 14x multiple.
Using a P/E instead of a DCF simply uses next year as the terminal value.
In many cases, most of the value is in the terminal multiple anyway, so a P/E shortcuts that process.


3) EV-to-EBITDA*

Same concept, but replacing terms that apply to the EV not just the equity.
4) Price-to-Revenue*

All performance ratios are ultimately proxies/shortcuts for DCFs.

But in practice, often the conditions cited above don't exist yet.

Price-to-revenue applies where there is not yet steady profitability, but there is enough clarity on revenue growth and bounded LT margin range.


5) Asset Coverage

For credit investing or other situations where there is tangible asset value underlying the investment.


6) Correlations / Pure Comparison

Macro variables, currencies, commodities, products, art, and others are often are priced simply as a function of their relationship to other major similar or analogous assets.


7) Marginal Cost (sometimes plus ROIC)

Commodities in oversupply price to the marginal cost of production.
More subtly, a similar construct applies to goods, services, FX, and other semi-commoditized markets.


8) Marginal Utility

Commodities in scarcity price to the marginal utility.


9) Value-per-User

Similar to price-to-revenue, a leading/proxy DCFs shortcut.


10) TAM capture

Also a several-steps-removed DCF shortcut.


11) Team / Value Creation

Can a team add value separate from current fundamentals.


12) Theoretical Models

Included for completeness, usually less practical. A prominent one in FX (and certain digital assets) is MV = PQ which relates money supply, velocity, and real activity levels to price levels.